*This notebook contains interactive elements. Please make sure your Python kernel and your notebook support interactivity.*

## The Wordle Game

**Wordle** is an English word game. You as a player, by default settings, has to guess a 5-letter English word within 6 attempts.

A guess has to be a 5-letter word. When a guess has been made, the game reveals to you how accurate your guess was, by marking your letters with colors:

- <span style="color: grey;">Grey</span> letters are the letters that **do not** appear in the correct word.
- <span style="color: goldenrod;">Yellow</span> letters are the letters that **do** appear in the correct word, however are placed at the **wrong** positions.
- <span style="color: darkgreen;">Green</span> letters are the letters that **do** appear in the correct word, and are placed at the **right** positions.

The game ends when you guess the correct hidden word within 6 attempts.

## The Wordle vs. Computer game

The rule of the **Wordle** game still applies, however, this time, you face a computer that can also make guesses. As you play the game, after every guess you have made, the computer reveals to you the guess it has made. The catch is, the computer does not reveal to you the letters it has entered, but the game tells you how accurate the computer's guess is, using the colors.

You lose when you have not guessed the correct word and the computer has already finished the game with its correct guess. Just like you, however, the computer is not always winning, and might lose the game.

In [42]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from enum import Enum
from IPython.display import display, clear_output
from nltk.corpus import words
import random
import ipycanvas
import ipyevents
import collections
import time

In [43]:
def get_word_list_of_word_length(length: int):
        return list(map(lambda word: word.lower(), filter(lambda word: len(word) == length, words.words())))

In [44]:
class CellVerdict(Enum):
    NONE = 0
    NOT_IN_WORD = 1
    IN_WORD = 2
    EXACT = 3

In [45]:
class Guesser:
    def __init__(self, sampleSpace, solution_word: str):
        self.sampleSpace = sampleSpace
        self.solution_word = solution_word

    def userAnswer(self, my_input, answer = None):
        if answer is None:
            answer = self.solution_word
        my_input = my_input.upper()
        result = ""
        commonList = []
        inputList = list(set(my_input))
        answerList = list(set(answer))
        temp = 0
        for i in inputList:
            for j in answerList:
                if i == j:
                    commonList.append(i)     

        my_result = [0] * len(my_input) # Initialize array

        for count in range(len(my_input)): # We want to iterate over both words to see if, at any index, the values are exactly the same
            if my_input[count] == answer[count]:
                my_result[count] = "Green" #If so, GREEN

        #if ((all(x == my_result[0] for x in my_result)) and my_result[0] == "Green"):
        #    temp = -1 # At this point, if our array is all GREEN, we can say it is the correct answer

        #for letter in my_input: # If not, continue to YELLOW 
            elif my_input[count] in answer:
                #index = my_input.index(letter)
                my_result[count] = "Yellow"

            else:
                pass

        my_result = ["Red" if x == 0 else x for x in my_result] # If not GREEN or YELLOW, then RED

        if (temp == -1):
            return ("Correct") # If our array was all GREEN, return CORRECT
        else:
            return my_result    
    
    def reduceSpace(self, firstGuess, itemsDelete, itemsWithin, itemsExact):

        iteration = self.userAnswer(firstGuess, self.solution_word)

        indicesRED = [i for i, x in enumerate(iteration) if x == "Red"]

        for item in indicesRED:
            itemsDelete.append(firstGuess[item])

        indicesYELLOW = [i for i, x in enumerate(iteration) if x == "Yellow"]

        for item in indicesYELLOW:
            itemsWithin.append(firstGuess[item])

        indicesGREEN = [i for i, x in enumerate(iteration) if x == "Green"]

        for item in indicesGREEN:
            itemsExact.append(firstGuess[item])

        ans1 = [ele for ele in self.sampleSpace if all(ch not in ele for ch in itemsDelete)]

        ans2 = [ele for ele in ans1 if all(ch in ele for ch in itemsWithin)]

        ans3 = [word for word in ans2 if all(firstGuess[i] == word[i] for i in indicesGREEN)]

        if ans3:
            return ans3, itemsDelete, itemsWithin, itemsExact
        elif ans2:
            return ans2, itemsDelete, itemsWithin, itemsExact
        else:
            return ans1, itemsDelete, itemsWithin, itemsExact

    def newGuess (self, newSpace):
        if not newSpace:
            return self.solution_word
        nextguessWord = random.choice(newSpace)
        return nextguessWord
    
    def n_guesses(self, n: int):
        firstGuess = random.choice(self.sampleSpace)
        yield firstGuess
        
        delete = []
        within = []
        exact = []
        itemsGuessed = []
        itemsGuessed.append(firstGuess)

        guess1 = firstGuess

        for i in range(n-1):
            SS1, delete1, within1, exact1 = self.reduceSpace(guess1, delete, within, exact)
            guess1 = self.newGuess(SS1)
            itemsGuessed.append(guess1)
            yield guess1

            if (guess1 == self.solution_word):
                break
        
        return itemsGuessed

In [46]:
class WordleGame:
    EMPTY_CHAR = ' '
    
    def __init__(self, word_list: list[str] = None):
        self.word_list = word_list if word_list is not None else ["apple"]
        self.solution_word: str = random.choice(self.word_list)
        self.lines: list[list[str]] = [[]]
        self.cell_verdicts: list[list[CellVerdict]] = [[]]
        self.max_attempts: bool = 6 # 0 or negative for unlimited attempts
        self.game_over: bool = False
        self.winning: bool = None
        
    def start():
        self.solution_word: str = random.choice(self.word_list)
        self.lines = [[]]
        self.cell_verdicts = [[]]
        self.game_over = False
        self.winning = None

    def abort():
        self.game_over = True

    def get_letter_count(self):
        return len(self.solution_word)

    def get_content_for_cell(self, row_idx: int, col_idx: int):
        if len(self.lines) == 0:
            return WordleGame.EMPTY_CHAR
        while row_idx < 0:
            row_idx = len(self.lines) + row_idx
        if len(self.lines) <= row_idx or len(self.lines[row_idx]) == 0:
            return WordleGame.EMPTY_CHAR
        while col_idx < 0:
            col_idx = len(self.lines[row_idx]) + col_idx
        if len(self.lines[row_idx]) <= col_idx:
            return WordleGame.EMPTY_CHAR
        return self.lines[row_idx][col_idx]
    
    def get_cell_verdict_for_cell(self, row_idx: int, col_idx: int):
        if len(self.cell_verdicts) == 0:
            return CellVerdict.NONE
        while row_idx < 0:
            row_idx = len(self.cell_verdicts) + row_idx
        if len(self.cell_verdicts) <= row_idx or len(self.cell_verdicts[row_idx]) == 0:
            return CellVerdict.NONE
        while col_idx < 0:
            col_idx = len(self.cell_verdicts[row_idx]) + col_idx
        if len(self.cell_verdicts[row_idx]) <= col_idx:
            return CellVerdict.NONE
        return self.cell_verdicts[row_idx][col_idx]
    
    def get_bg_text_colors_for_cell(self, row_idx: int, col_idx: int):
        cell_verdict = self.get_cell_verdict_for_cell(row_idx, col_idx)

        if cell_verdict == CellVerdict.NONE:
            return ("white", "black")
        if cell_verdict == CellVerdict.NOT_IN_WORD:
            return ("gray", "white")
        if cell_verdict == CellVerdict.IN_WORD:
            return ("goldenrod", "white")
        if cell_verdict == CellVerdict.EXACT:
            return ("darkgreen", "white")
        
        return ("black", "white")
    
    def give_verdict_for(self, letter: str, col: int):
        letter = letter.lower()
        while col < 0:
            col = self.get_letter_count() + col
        if len(self.solution_word) <= col:
            return CellVerdict.NONE
        if self.solution_word[col] == letter:
            return CellVerdict.EXACT
        if letter in self.solution_word:
            return CellVerdict.IN_WORD
        return CellVerdict.NOT_IN_WORD
    
    def give_verdicts_for(self, word: str):
        word = word.lower()
        solution_word_letter_counts = collections.Counter(self.solution_word.lower())
        results = [CellVerdict.NONE for _ in range(len(self.solution_word))]
        for idx, character in enumerate(word):
            if self.solution_word[idx] == character:
                results[idx] = CellVerdict.EXACT
                solution_word_letter_counts[character] -= 1
        for idx, character in enumerate(word):
            if results[idx] == CellVerdict.EXACT:
                continue
            if character in solution_word_letter_counts:
                if solution_word_letter_counts[character] > 0:
                    results[idx] = CellVerdict.IN_WORD
                    solution_word_letter_counts[character] -= 1
                else:
                    results[idx] = CellVerdict.NOT_IN_WORD
            else:
                results[idx] = CellVerdict.NOT_IN_WORD
        return results
    
    def check_lose(self):
        if self.max_attempts <= 0:
            return False
        return len(self.lines) >= self.max_attempts and not all(map(lambda cell_verdict: cell_verdict == CellVerdict.EXACT, self.cell_verdicts[-1]))
    
    def check_win(self):
        return all(map(lambda cell_verdict: cell_verdict == CellVerdict.EXACT, self.cell_verdicts[-1]))
    
    def process_input_key(self, key: str):
        if self.game_over:
            return
        current_line = self.lines[-1]
        current_cell_verdicts_row = self.cell_verdicts[-1]
        if key == "Backspace":
            if len(current_line) == 0:
                return
            current_line.pop()
        if key == "Enter":
            if len(current_line) >= self.get_letter_count():
                current_cell_verdicts_row.extend(self.give_verdicts_for(''.join(current_line)))
                lose = self.check_lose()
                if lose:
                    self.winning = False
                    self.game_over = True
                win = self.check_win()
                if win:
                    self.winning = True
                    self.game_over = True
                if not lose and not win:
                    self.lines.append([])
                    self.cell_verdicts.append([])
            return
        if len(current_line) >= self.get_letter_count():
            return
        if len(key) == 1:
            if ord('A') <= ord(key) <= ord('Z') or ord('a') <= ord(key) <= ord('z'):
                current_line.append(key.upper())

In [47]:
class WordleRenderer:
    def __init__(self, game: WordleGame, canvas: ipycanvas.Canvas = None, do_render_text: bool = True):
        self.game = game
        self.canvas = ipycanvas.Canvas()
        self.cell_size = 64
        self.cell_gap = 4
        self.font = "32px monospace"
        self.do_render_text = do_render_text
    
    def generate_canvas(self):
        with ipycanvas.hold_canvas(self.canvas):
            self.canvas.width = self.game.get_letter_count() * self.cell_size + max(self.game.get_letter_count() - 1, 0) * self.cell_gap
            self.canvas.height = len(self.game.lines) * self.cell_size + max(len(self.game.lines) - 1, 0) * self.cell_gap
            self.canvas.font = self.font
            self.canvas.text_align = "center"
            self.canvas.text_baseline = "middle"
            for row_idx in range(len(self.game.lines)):
                for col_idx in range(self.game.get_letter_count()):
                    top_left_x = col_idx * self.cell_size + col_idx * self.cell_gap
                    top_left_y = row_idx * self.cell_size + row_idx * self.cell_gap
                    bg_color, text_color = self.game.get_bg_text_colors_for_cell(row_idx, col_idx)
                    content = self.game.get_content_for_cell(row_idx, col_idx)
                    self.canvas.fill_style = bg_color
                    self.canvas.fill_rect(top_left_x, top_left_y, self.cell_size)
                    if self.do_render_text:
                        self.canvas.fill_style = text_color
                        self.canvas.fill_text(content, top_left_x + self.cell_size/2, top_left_y + self.cell_size/2)
        return self.canvas
    
    def render(self):
        self.generate_canvas()
        display(self.canvas)

In [48]:
class WordleGameStatus:
    def __init__(self, game: WordleGame):
        self.game = game
        self.status = widgets.Label()

    def update(self):
        max_attempts = self.game.max_attempts
        if max_attempts <= 0:
            max_attempts = "∞"
        
        text: str = f"Current attempt: {len(self.game.lines)}/{max_attempts}"
        color: str = None
        
        if self.game.game_over and self.game.winning is not None:
            if self.game.winning:
                text = "YOU WIN!"
                color = "green"
            else:
                text = f"YOU LOSE... The word is \"{self.game.solution_word}\""
                color = "red"
        
        self.status.value = text
        if color:
            self.status.style.text_color = color

    def render(self):
        self.update()
        display(self.status)

In [49]:
word_length = 5
word_list = get_word_list_of_word_length(word_length)

user_game = WordleGame(word_list)
user_game_renderer = WordleRenderer(user_game)
user_game_status = WordleGameStatus(user_game)

computer_game = WordleGame([user_game.solution_word])
computer_game_renderer = WordleRenderer(game=computer_game, do_render_text=False)
computer_game_status = WordleGameStatus(computer_game)
guesser = Guesser(word_list, user_game.solution_word)
guess_gen = guesser.n_guesses(user_game.max_attempts)


In [50]:
def handle_keydown(evt):
    if user_game.game_over or computer_game.game_over:
        return
    k = evt["key"]
    user_game.process_input_key(k)
    if len(user_game.lines) > len(computer_game.lines):
        try:
            next_guess = next(guess_gen)
        except StopIteration:
            pass
        else:
            for character in next_guess:
                computer_game.process_input_key(character)
            computer_game.process_input_key("Enter")
    user_game_renderer.generate_canvas()
    computer_game_renderer.generate_canvas()
    user_game_status.update()
    computer_game_status.update()
    if computer_game.game_over and computer_game.winning and not user_game.winning:
        user_game_status.status.value = f"You lose against the computer. The word is \"{computer_game.solution_word}\""

event_keydown_handler = ipyevents.Event(source=user_game_renderer.canvas, watched_events=["keydown"], prevent_default_action=True)
event_keydown_handler.on_dom_event(handle_keydown)

In [51]:
user_game_renderer.generate_canvas()
computer_game_renderer.generate_canvas()
user_game_status.update()
computer_game_status.update()

display(
    widgets.HBox([
        widgets.VBox([
            user_game_renderer.canvas,
            user_game_status.status
        ]),
        widgets.VBox([
            widgets.Label("<= You are playing on the left side", layout={"margin": "0 3rem"}),
            widgets.Label("The computer is playing on the right side =>", layout={"margin": "0 3rem"}),
        ]),
        widgets.VBox([
            computer_game_renderer.canvas,
            computer_game_status.status
        ]),
    ])
)